In [1]:
# Perform the initialization and imports
import sys
import pickle
import re
import os
import csv
import argparse
import math

from string import ascii_lowercase
from collections import Counter, defaultdict
from operator import itemgetter

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from Bio import SeqIO, AlignIO
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import IUPAC
from Bio.Seq import Seq
from Bio.Emboss.Applications import NeedleallCommandline

# Demand Python 3.
if sys.version_info[0] < 3:
    print("Python 3 is required, but you are using Python %i.%i.%i") % (
        sys.version_info[0], sys.version_info[1], sys.version_info[2])
    sys.exit(1)

In [2]:
# Retrieve the specific functions from ind and proteins.py
indels_path="/home/maya/InDelScanner"  # /PATH/TO/InDelScanner
if indels_path not in sys.path:
    sys.path.append(indels_path)
#from indels.ind import trim_read, findEnds, endMatch, findGap, gapAlign

from ipynb.fs.defs.Library_diversity import convert_variant_to_dict

os.chdir("/mnt/c/Users/Maya/Dropbox/mek_results")

with open('kinases_counter.pickle', 'rb') as f:
    mek = pickle.load(f)

with open('df_pos_filtered.pickle', 'rb') as f:
    df_pos = pickle.load(f)

pos = df_pos.to_dict()

In [3]:
# Set general restrictions stemming from SpliMLib library design
aa_2 = ['A', 'Δ']
aa_12 = ['A','G','P','Y','D','K','M','V','I','L','F','W']
aa_13 = aa_12 +  ['Δ']
pos_aa = {'6': aa_12, '9': aa_12, '11': aa_12, '13': aa_12, '7a': aa_13, '8a': aa_2}

In [4]:
active_ls = df_pos.index.tolist()
active = {short : convert_variant_to_dict(short) for short in active_ls}
valid_pos = ['6', '7a', '8a', '9', '11', '13']

data = {}
for short, m_to_pos in active.items():
    if len(m_to_pos) != len(valid_pos):
        continue
    else:
        data[short] = [m_to_pos[i] for i in valid_pos]

factors = pd.DataFrame.from_dict(data, orient='index', columns=valid_pos).reset_index(drop=False)

In [5]:
factors.to_pickle('mkk_factors.gzip')

## GraphML

In [6]:
import random

In [7]:
test = random.sample(active.items(), 1000)
test_dict = {}
for i in range(len(test)):
    test_dict[test[i][0]] = test[i][1]

In [8]:
# hamming distance = 0 if the aa matches at all positions
def hamming_distance(s1, s2, pos_aa):
    d = 0
    if (len(s1) != len(pos_aa)) or (len(s2) != len(pos_aa)):
        d += 2

    for p in pos_aa.keys():
        if s1[p] != s2[p]:
            d += 1
    return d

In [9]:
# make a list of edges
def list_edge_nodes(active, pos_aa):
    node_pairs = []
    for short1, s1 in active.items():
        for short2, s2 in active.items():
            h_dis = hamming_distance(s1, s2, pos_aa)
            if h_dis == 1:
                node_pairs.append((short1, short2))
    return node_pairs

In [10]:
header = '<?xml version="1.0" encoding="utf-8"?>'
graphml_open = '<graphml xmlns="http://graphml.graphdrawing.org/xmlns" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" \
    xsi:schemaLocation="http://graphml.graphdrawing.org/xmlns  http://graphml.graphdrawing.org/xmlns/1.0/graphml.xsd">'
graph_open = '<graph edgedefault="undirected">'

close = '</graph>\n</graphml>'

In [11]:
# GRAPHML functions

# create nodes
def gml_write_node(short, file):
    print('<node id=\"{node}"/>'.format(node=short), sep="", file=file)

def gml_list_edge_strings(node_pairs):
    edge_strings = []
    for pair in node_pairs:
        edge_id = '-'.join(pair)
        one_edge = '<edge id="{id}" source="{s}" target="{t}"/>'.format(
            id=edge_id, s=pair[0], t=pair[1])
        edge_strings.append(one_edge)
    return edge_strings

Now write the graph file.

In [14]:
def write_graphml(graph_dict, filename):
    with open (filename, 'w') as f:
        # print the GRAPHML header definitions
        print(header, graphml_open, graph_open, file=f, sep='\n')

        # write nodes
        for short in graph_dict.keys():
            gml_write_node(short, f)

        # write edges
        node_pairs = list_edge_nodes(graph_dict, pos_aa)
        edge_strings = gml_list_edge_strings(node_pairs)
        f.writelines(edge_strings)

        # end the graph and graphml
        print(close, file=f)
    
    return node_pairs

In [15]:
test_edges = write_graphml(test_dict, 'test.graphml')

In [17]:
active_edges = write_graphml(active, 'active.graphml')

## Sequence similarity network analysis with NetworkX

In [ ]:
import networkx as nx

In [ ]:
g_ham = nx.read_graphml('active.graphml')

In [ ]:
# AVERAGE NODE DEGREE
g_ham.size() / g_ham.order()

In [ ]:
ham_components = list(g_ham.subgraph(c) for c in nx.connected_components(g_ham))

In [ ]:
len(ham_components)

It is justified to only examine the largest connected subgraph, since the next subgraphs have very few nodes: 5 (2x), 4 (4x), 3 (12x) or less.

In [ ]:
component_lens = {i: len(ham_components[i]) for i in range(len(ham_components))} # get the number of nodes in the largest subgraphs

sort_components = sorted(component_lens.items(), key=lambda x: x[1], reverse=True)

for i in sort_components:
	print(i[0], i[1])

In [ ]:
G = ham_components[0]

In [ ]:
G.size()/G.order()

In [ ]:
degree_sequence = sorted([d for n, d in G.degree()], reverse=True)  # degree sequence
# print "Degree sequence", degree_sequence
degreeCount = Counter(degree_sequence)
deg, cnt = zip(*degreeCount.items())

fig, ax = plt.subplots()
plt.bar(deg, cnt, width=0.80, color='b')

plt.title("Largest connected subgraph")
plt.ylabel("Count")
plt.xlabel("Degree")
plt.show()

In [ ]:
degree_sequence = sorted([d for n, d in g_ham.degree()], reverse=True)  # degree sequence
# print "Degree sequence", degree_sequence
degreeCount = Counter(degree_sequence)
deg, cnt = zip(*degreeCount.items())

fig, ax = plt.subplots()
plt.bar(deg, cnt, width=0.80, color='b')

plt.title("Full Hamming distance graph")
plt.ylabel("Count")
plt.xlabel("Degree")
plt.show()

This is looking promising. At this point, take the largest subgraph into Gephi to build a good visualisation.

In [ ]:
nx.write_graphml(G, 'connected.graphml')

In [ ]:
G.degree('6L/7aI/8aA/9L/11F/13M')

In [ ]:
nx.set_node_attributes(G, 'degree', G.degree)

In [ ]:
deg = nx.get_node_attributes(G,'degree')

Much like with Cytoscape, this ends up crashing/stops responding because the graph is too big. However, I think it's going in the right direction. Let's add an attribute to the graph for every note to save the original degree and then kick out the less connected nodes.

In [ ]:
G = nx.read_graphml('connected.graphml')

In [ ]:
H = nx.Graph()
# copy over the nodes from G
for n in G.nodes():
    H.add_node(n, deg=G.degree[n])
for e in G.edges():
    H.add_edge(e[0], e[1])

Now let's start experimenting with pruning.

https://stackoverflow.com/questions/18381187/functions-for-pruning-a-networkx-graph

One option is to try pruning nodes according to betweeness centrality. First, calculate the betweeness centrality for all nodes, identifying the ones that are 'hubs': appear in many shortest paths between nodes. Plot a distribution on betcen to get an idea of the values.

In [ ]:
# Closeness centrality
clo_cen = nx.closeness_centrality(H)

In [ ]:
with open('H_closeness_centrality.pickle', 'wb') as f:
    pickle.dump(clo_cen, f)

In [ ]:
# Betweenness centrality
bet_cen = nx.betweenness_centrality(H)

In [ ]:
with open('H_betweenness_centrality.pickle', 'wb') as f:
    pickle.dump(clo_cen, f)

In [ ]:
eig_cen = nx.eigenvector_centrality(H)

with open('H_eigenvector_centrality.pickle', 'wb') as f:
    pickle.dump(eig_cen, f)

In [ ]:
nx.set_node_attributes(H, clo_cen, 'closeness')
nx.set_node_attributes(H, bet_cen, 'betweenness')
nx.set_node_attributes(H, eig_cen, 'eigenvector')

In [ ]:
nx.write_graphml(H, 'connected_annotated.graphml')

In [ ]:
H.order()

In [ ]:
%time layt_sp = nx.spectral_layout(H)

In [ ]:
fig = plt.figure()
axs = fig.add_subplot(111, aspect='equal')
axs.axis('off')
nx.draw_networkx_edges(H, layt_sp)
nx.draw_networkx_nodes(H, layt_sp)
plt.show()

In [ ]:
degree_dict = dict(H.degree(H.nodes()))
sorted_degree = sorted(degree_dict.items(), key=itemgetter(1), reverse=True)
print("Top 20 nodes by degree:")
for d in sorted_degree[:20]:
    print(d)

In [ ]:
nx.diameter(H)